In [1]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
import xgboost as xgb

%matplotlib inline

In [2]:
test = pd.read_csv('../data/raw/X_test.csv')
x_train = pd.read_csv('../data/raw/X_train.csv')
y_train = pd.read_csv('../data/raw/y_train.csv')

In [3]:
x_train.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410


In [4]:
y_train.head()

,series_id,group_id,surface
0,0,13,fine_concrete
1,1,31,concrete
2,2,20,concrete
3,3,31,concrete
4,4,22,soft_tiles


In [5]:
x_train.columns

Index(['row_id', 'series_id', 'measurement_number', 'orientation_X',
       'orientation_Y', 'orientation_Z', 'orientation_W', 'angular_velocity_X',
       'angular_velocity_Y', 'angular_velocity_Z', 'linear_acceleration_X',
       'linear_acceleration_Y', 'linear_acceleration_Z'],
      dtype='object')

In [29]:
import math 

def quarternion_to_euler(x, y, z, w):
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        X = math.degrees(math.atan2(t0, t1))

        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        Y = math.degrees(math.asin(t2))

        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        Z = math.degrees(math.atan2(t3, t4))

        return X, Y, Z

In [32]:
x = x_train.iloc[0:1, 3][0]
y = x_train.iloc[0:1, 4][0]
z = x_train.iloc[0:1, 5][0]
w = x_train.iloc[0:1, 6][0]
ret = quarternion_to_euler(x, y, z, w)
list(ret)

[162.90750255124766, -1.413350252799204, 80.0229529552707]

In [48]:
quarternion_cols = ['orientation_X', 'orientation_Y', 'orientation_Z', 'orientation_W']
quarternion = x_train[quarternion_cols]
euler = pd.DataFrame(list(quarternion.apply(lambda x: quarternion_to_euler(x.orientation_X, x.orientation_Y, x.orientation_Z, x.orientation_W), axis = 1)))
euler.columns = ['x', 'y', 'z']

In [49]:
x_train = x_train.join(euler, how='outer')

In [53]:
x_train.groupby(['series_id'])['x','y','z'].agg(['min','mean','max'])

x                                 y                      \
                  min        mean         max       min      mean       max   
series_id                                                                     
0          162.732239  162.819374  162.945400 -1.477940 -1.434513 -1.350003   
1          162.583377  162.727412  162.853546 -0.691722 -0.594074 -0.521926   
2          162.923749  163.036775  163.138194 -0.729595 -0.698720 -0.675807   
3          162.797712  163.051028  163.415917 -0.899404 -0.865580 -0.839090   
4          162.788205  162.859936  162.934193 -0.583948 -0.561119 -0.536152   
5          162.531204  162.684254  162.805787 -1.433460 -1.342600 -1.233625   
6          162.723416  162.804176  162.910439 -1.131590 -1.114104 -1.094863   
7          162.839770  163.020355  163.205452 -0.747641 -0.643650 -0.560573   
8          162.982570  163.010019  163.052475 -0.936103 -0.889336 -0.824011   
9          162.482647  162.644603  162.781357 -1.126803 -1.061640 -0.996610   
10         163.083866  163.190164  163.325843 -0.477606 -0.419186 -0.355873   
11         162.732594  162.947044  163.140518 -1.570063 -1.508584 -1.455579   
12         162.700194  162.881433  163.102108 -1.070910 -0.933192 -0.789041   
13         162.503902  162.574126  162.649875 -1.076058 -0.886756 -0.784530   
14         161.881053  162.398698  162.881155 -0.621665 -0.560115 -0.444385   
15         162.987354  163.116085  163.310843 -1.041907 -0.940441 -0.845505   
16         162.323629  162.671229  163.034980 -1.394952 -0.833948 -0.427438   
17         162.800446  162.875867  162.940912 -0.888407 -0.874599 -0.862959   
18         162.777891  163.126075  163.401389 -0.915282 -0.842531 -0.729699   
19         162.761907  162.793998  162.832288 -0.827950 -0.767532 -0.707348   
20         162.754914  162.832995  162.942451 -0.903398 -0.841632 -0.761840   
21         163.080959  163.089753  163.096202 -1.004668 -0.996208 -0.983662   
22         162.781327  162.787751  162.791267 -0.497114 -0.494811 -0.491535   
23         162.611566  162.769939  162.954534 -0.598700 -0.464300 -0.393771   
24         162.771728  162.796768  162.818914 -0.694703 -0.673827 -0.645920   
25         162.687184  162.746302  162.781588 -0.858945 -0.840940 -0.804479   
26         163.196395  163.247239  163.284368 -0.923393 -0.898235 -0.857422   
27         162.886655  162.949343  163.039759 -0.972011 -0.935834 -0.900197   
28         162.615459  162.738317  162.876084 -1.281168 -1.153055 -1.041364   
29         162.929784  162.997589  163.042794 -0.940535 -0.912638 -0.898775   
...               ...         ...         ...       ...       ...       ...   
3780       162.994817  163.139568  163.268691 -0.944558 -0.831550 -0.720900   
3781       162.373115  162.586879  162.773556 -0.943097 -0.709981 -0.492772   
3782       162.856053  162.961078  163.066759 -1.223074 -1.099617 -0.958356   
3783       162.441281  162.535904  162.661788 -0.955985 -0.893294 -0.834517   
3784       162.188477  162.372266  162.525091 -0.601730 -0.536829 -0.463866   
3785       162.611635  162.679091  162.801218 -0.955211 -0.894649 -0.841300   
3786       163.148561  163.255361  163.368735 -0.536069 -0.450735 -0.391785   
3787       162.419056  162.517700  162.608592 -0.746474 -0.679965 -0.609094   
3788       166.470262  166.575855  166.676997 -1.201151 -1.165801 -1.090290   
3789       163.024438  163.076870  163.100842 -0.794680 -0.769940 -0.738351   
3790       162.633118  162.841789  163.152622 -0.775095 -0.639180 -0.557339   
3791       161.616135  162.550833  163.433267 -0.704063 -0.616482 -0.409903   
3792       166.436501  166.560851  166.720570 -1.015073 -0.972645 -0.912507   
3793       162.738220  162.869869  162.957099 -0.640610 -0.605965 -0.569772   
3794       162.875240  163.000445  163.201659 -0.791481 -0.693783 -0.583288   
3795       162.997525  163.209643  163.423594 -0.591039 -0.504099 -0.385299   
3796       162.843419  162.869728  162.894886 -0.799809 -0.768221 -0.7